In [1]:
import os
import cv2
from PIL import Image
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt



In [2]:
from keras import *

# image processing
from keras.preprocessing import image as image_utils
from keras.preprocessing.image import img_to_array

# build your own nets
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import RMSprop
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU



Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:496: FutureWarn

In [3]:
train_image_files_path = "Data/brickImages/train"
valid_image_files_path = "Data/brickImages/valid"
test_image_files_path = "Data/brickImages/test"

In [4]:
brick_list = [ '11214 Bush 3M friction with Cross axle','3004 Brick 1x2','3024 Plate 1x1','3673 Peg 2M', 
              '18651 Cross Axle 2M with Snap friction','3005 Brick 1x1','3040 Roof Tile 1x2x45deg','3713 Bush for Cross Axle',
              '2357 Brick corner 1x2x2','3022 Plate 2x2','3069 Flat Tile 1x2','3794 Plate 1X2 with 1 Knob','3003 Brick 2x2',
              '3023 Plate 1x2','32123 half Bush','6632 Technic Lever 3M' ]

print(brick_list[0])


11214 Bush 3M friction with Cross axle


In [5]:
im = Image.open(train_image_files_path + "/" + brick_list[0] + "/201706171006-0001.png")
img_width, img_height = im.size

output_n = len(brick_list)
size = 50
channels = 3
batch_size = 16

train_data_gen = ImageDataGenerator(
    rescale = 1 / 255,
    rotation_range = 40,
)

valid_data_gen = ImageDataGenerator(
    # validation data shouldn't be augmented
    rescale = 1 / 255
)

train_image_array_gen = train_data_gen.flow_from_directory(
    train_image_files_path,
    target_size = (img_width, img_height),
    class_mode = 'categorical',
    classes = brick_list,
    color_mode = 'rgb', 
    batch_size = batch_size,
    seed = 42)

valid_image_array_gen = valid_data_gen.flow_from_directory(
    valid_image_files_path,
    target_size = (img_width, img_height),
    class_mode = 'categorical',
    classes = brick_list,
    color_mode = 'rgb', 
    batch_size = batch_size,
    seed = 42)

Found 6379 images belonging to 16 classes.
Found 6379 images belonging to 16 classes.


In [6]:
epochs = 3
train_samples = train_image_array_gen.n
valid_samples = valid_image_array_gen.n
print(train_samples, valid_samples)

model = Sequential()

model.add(Conv2D(64, (3, 3), padding = "valid", input_shape = (img_width, img_height, channels)))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3), padding = "same"))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha = 0.5))

model.add(Conv2D(16, (3, 3), padding = "same"))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha = 0.5))

model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.125))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(output_n))
model.add(Activation('softmax'))

print(model.summary())

6379 6379
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 198, 198, 64)      1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 198, 198, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 198, 198, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 198, 198, 32)      18464     
_________________________________________________________________
batch_normalization_2 (Batch (None, 198, 198, 32)      128       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 198, 198, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 198, 198

In [7]:
model.compile(loss = 'categorical_crossentropy', 
              optimizer = RMSprop(lr = 0.0001, decay = 1e-6),
              metrics = ['accuracy'])

In [ ]:
history = model.fit_generator(
    train_image_array_gen,
    steps_per_epoch = int(train_samples / batch_size), 
    epochs = epochs, 
    validation_data = valid_image_array_gen,
    validation_steps = int(valid_samples / batch_size),
    verbose = 1
)

Epoch 1/3
 18/398 [>.............................] - ETA: 3:49:46 - loss: 12.1747 - acc: 0.1701

In [ ]:
!cd test_image_files_path && wget https://images-na.ssl-images-amazon.com/images/I/512rSmEUxBL._SL1001_.jpg

test_images = !find $test_image_files_path -type f -name "*.jpg"

def classify_image_model(image, classes=classes):
    img = cv2.imread(image)        
    b,g,r = cv2.split(img)       
    img = cv2.merge([r,g,b])     
    plt.imshow(img)
    plt.xticks([]), plt.yticks([])
    plt.show()
    
    image = image_utils.load_img(image, target_size=(img_width, img_height))
    image = image_utils.img_to_array(image)

    image = np.expand_dims(image, axis=0)

    # scale pixels between -1 and 1, sample-wise
    image /= 255.
        
    prediction = model.predict(image)
    
    pred = prediction.argmax()

    for k, v in classes.items():
        if (v == pred):
            pred_label = k
        
    proba = prediction.max()
    
    print("Predicted class: " + pred_label + " with probability " + str(proba*100) + "%")

print(test_images[0])